In [1]:
%pip install fastapi uvicorn nest_asyncio wikipedia google-api-python-client


^C
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError
import nest_asyncio
import uvicorn

# Allow FastAPI to run in Jupyter
nest_asyncio.apply()

app = FastAPI()

# CORS config
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173", "http://127.0.0.1:5173"], 
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.get("/")
def welcome():
    return {"message": "API is running!"}

def search_wikipedia(query, level):
    try:
        page = wikipedia.page(query)
        summary = page.summary
        wiki_url = page.url

        sentences = summary.split(". ")
        beginner = ". ".join(sentences[:2]) + "."
        intermediate = ". ".join(sentences[:4]) + "."
        advanced = summary

        if level == "beginner":
            return beginner, wiki_url
        elif level == "intermediate":
            return intermediate, wiki_url
        elif level == "advanced":
            return advanced, wiki_url
        else:
            return "Invalid level. Choose beginner, intermediate, or advanced.", ""

    except DisambiguationError as e:
        return f"Multiple topics found for '{query}'. Please be more specific.", ""
    except PageError:
        return f"No Wikipedia page found for '{query}'.", ""
    except Exception as e:
        return f"An error occurred: {str(e)}", ""

@app.get("/ask/")
def ask_question(question: str, level: str = "beginner"):
    answer, wiki_url = search_wikipedia(question, level)

    return {
        "question": question,
        "level": level,
        "answer": answer,
        "videos": f"https://www.youtube.com/results?search_query={question}+{level}",
        "pdf_search": f"https://www.google.com/search?q={question}+{level}+filetype:pdf",
        "wikipedia_link": wiki_url
    }

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=3000)
